In [1]:
import pandas as pd 
import numpy as np
#methods
from surprise import SVD, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from mlxtend.frequent_patterns import apriori, association_rules
#other
import itertools

In [25]:
df_users = pd.read_csv('Users_games_dataset.csv')
df_details = pd.read_csv('games_details_dataset.csv')

#### Association rule with apriori

In [3]:
df_users["Played"] = [True for i in range(df_users.shape[0])]
df_user_features = df_users.pivot(index='User_id',columns='Game_id',values='Played').fillna(False)
df_user_features.head()

Game_id,10,20,30,40,50,60,70,80,130,220,...,423880,480490,581300,608580,752180,764050,828860,860270,890400,961620
User_id,,,,,,,,,,,,,,,,,,,,,
5250,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
76767,True,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
86540,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
144736,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
181212,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
freq_items = apriori(df_user_features.astype(np.uint8), min_support=0.004, use_colnames=True, verbose=1)

C:\Users\yiann\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Processing 1547 combinations | Sampling itemset size 765


In [5]:
freq_items['length'] = freq_items['itemsets'].apply(lambda x: len(x))

In [6]:
freq_items.head()

,support,itemsets,length
0,0.053992,(10),1
1,0.008935,(30),1
2,0.011027,(70),1
3,0.023384,(80),1
4,0.004373,(130),1


In [7]:
freq_items.tail()

,support,itemsets,length
6787,0.004278,"(4000, 304930, 440, 730, 570, 218620)",6
6788,0.004087,"(4000, 304930, 230410, 440, 730, 570)",6
6789,0.004468,"(4000, 304930, 301520, 440, 730, 570)",6
6790,0.004468,"(105600, 4000, 620, 72850, 440, 730)",6
6791,0.004183,"(105600, 8930, 620, 72850, 440, 730)",6


In [8]:
rules = association_rules(freq_items, metric="confidence", min_threshold=0.001)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

In [9]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
22342,"(240, 440, 222880)",(730),0.004848,0.130894,0.004848,1.0,7.639797,0.004213,inf
23280,"(240, 550, 222880)",(730),0.004373,0.130894,0.004373,1.0,7.639797,0.003800,inf
55874,"(4000, 550, 222880, 440)",(730),0.004563,0.130894,0.004563,1.0,7.639797,0.003966,inf
7420,"(630, 55230)",(440),0.004658,0.220817,0.004658,1.0,4.528627,0.003629,inf
22690,"(105600, 240, 42910)",(440),0.004468,0.220817,0.004468,1.0,4.528627,0.003481,inf


In [10]:
rules.tail()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
42610,(570),"(24240, 730, 4000)",0.460171,0.007605,0.004087,0.008882,1.168044,0.000588,1.001289
13661,(570),"(218620, 271590)",0.460171,0.007700,0.004087,0.008882,1.153623,0.000544,1.001193
3516,(570),"(220, 22380)",0.460171,0.007985,0.004087,0.008882,1.112423,0.000413,1.000906
1051,(570),(259080),0.460171,0.008175,0.004087,0.008882,1.086552,0.000326,1.000714
1017,(570),(220200),0.460171,0.008555,0.004087,0.008882,1.038261,0.000151,1.000330


In [11]:
print("The number of rows is {}.".format(rules.shape[0]))

The number of rows is 68308.


In [12]:
new_index = [ i for i in range(rules.shape[0])]
rules.index = new_index

In [13]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(240, 440, 222880)",(730),0.004848,0.130894,0.004848,1.0,7.639797,0.004213,inf
1,"(240, 550, 222880)",(730),0.004373,0.130894,0.004373,1.0,7.639797,0.003800,inf
2,"(4000, 550, 222880, 440)",(730),0.004563,0.130894,0.004563,1.0,7.639797,0.003966,inf
3,"(630, 55230)",(440),0.004658,0.220817,0.004658,1.0,4.528627,0.003629,inf
4,"(105600, 240, 42910)",(440),0.004468,0.220817,0.004468,1.0,4.528627,0.003481,inf


In [14]:
small_conf = []
for i in range(rules.shape[0]):
    if float(rules.iloc[i]["confidence"]) < 0.5: #at the begging 0.2, but we need more speed for the iterations in the next step
        small_conf.append(i)
rules = rules.drop(labels=small_conf, axis=0)

In [15]:
print("The number of remaining rows is {}.".format(rules.shape[0]))

The number of remaining rows is 18463.


In [16]:
rules.tail()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
18458,"(400, 240)",(570),0.013118,0.460171,0.006559,0.5,1.086552,0.000522,1.079658
18459,"(500, 550)",(570),0.013688,0.460171,0.006844,0.5,1.086552,0.000545,1.079658
18460,"(730, 202970)",(570),0.008555,0.460171,0.004278,0.5,1.086552,0.000341,1.079658
18461,"(4000, 221100)",(570),0.009506,0.460171,0.004753,0.5,1.086552,0.000379,1.079658
18462,"(227940, 236390)",(570),0.009506,0.460171,0.004753,0.5,1.086552,0.000379,1.079658


In [17]:
maxim = 0
for i in range(rules.shape[0]):
    if len(rules.iloc[i]["antecedents"]) > maxim:
        maxim = len(rules.iloc[i]["antecedents"])
print("The maximum lentgh of right part of the rule is {}.".format(maxim))

The maximum lentgh of right part of the rule is 5.


In [18]:
user_games = [440, 500, 570, 550]

In [19]:
all_combinations = []
for i in range(2,maxim+1):
    all_combinations += list(itertools.combinations(user_games, i))

We will find all the recommended games based on the rules.

In [20]:
recommend = []
for game in user_games:
    for i in range(rules.shape[0]):
        antecedents_list = list(rules.iloc[i]["antecedents"])
        antecedents_list.sort()
        if antecedents_list == [game]:
            print(antecedents_list,"==>",list(rules.iloc[i]["consequents"]))
            recommend+=list(rules.iloc[i]["consequents"])
for comb in all_combinations:
    for i in range(rules.shape[0]):
        antecedents_list = list(rules.iloc[i]["antecedents"])
        antecedents_list.sort()
        comb_list = list(comb)
        comb_list.sort()
        if antecedents_list == comb_list:
            print(antecedents_list,"==>",list(rules.iloc[i]["consequents"]))
            recommend+=list(rules.iloc[i]["consequents"])

[500] ==> [550]
[500] ==> [440]
[500] ==> [440, 550]
[550] ==> [440]
[440, 500] ==> [550]
[440, 500] ==> [620]
[440, 500] ==> [240]
[440, 500] ==> [400]
[440, 500] ==> [730]
[440, 500] ==> [570]
[440, 570] ==> [730]
[440, 550] ==> [730]
[440, 550] ==> [570]
[500, 570] ==> [550]
[500, 570] ==> [440]
[500, 570] ==> [440, 550]
[500, 570] ==> [730]
[500, 570] ==> [620]
[500, 570] ==> [440, 730]
[500, 570] ==> [240]
[500, 570] ==> [440, 620]
[500, 570] ==> [240, 440]
[500, 570] ==> [730, 550]
[500, 570] ==> [620, 550]
[500, 570] ==> [49520]
[500, 570] ==> [400]
[500, 570] ==> [240, 550]
[500, 570] ==> [440, 730, 550]
[500, 570] ==> [240, 550, 440]
[500, 570] ==> [72850]
[500, 550] ==> [440]
[500, 550] ==> [620]
[500, 550] ==> [730]
[500, 550] ==> [240]
[500, 550] ==> [400]
[500, 550] ==> [570]
[550, 570] ==> [440]
[550, 570] ==> [730]
[550, 570] ==> [440, 730]
[440, 500, 570] ==> [550]
[440, 500, 570] ==> [730]
[440, 500, 570] ==> [620]
[440, 500, 570] ==> [240]
[440, 500, 570] ==> [730, 55

We have to remove duplicates from the list.

In [21]:
recommend = list(dict.fromkeys(recommend))

Finally, we want to remove games that user already likes.

In [22]:
remove_rec = []
for rec in recommend:
    if user_games.count(rec)>0:
        remove_rec.append(rec)
for rec in remove_rec:
    recommend.remove(rec)

In [23]:
print(recommend)

[620, 240, 400, 730, 49520, 72850, 630, 220]


In [26]:
for i in range(df_details.shape[0]):
    if recommend.count(df_details.iloc[i]['appid'])>0:
        print(df_details.iloc[i]['appid'],"=>",df_details.iloc[i]['name'])

220 => Half-Life 2
240 => Counter-Strike Source
400 => Portal
620 => Portal 2
630 => Alien Swarm
730 => Counter-Strike Global Offensive
49520 => Borderlands 2
72850 => The Elder Scrolls V Skyrim


## Matrix Factorization using SVD

In [27]:
df_users.head()

,User_id,Game,Hours-played,Game_id,Game_genre,Game_category,Game_playtime,Game_positive_votes,rating
0,151603712,The Elder Scrolls V Skyrim,273.0,72850,violent,no_multiplayer,more,positive,5
1,151603712,Fallout 4,87.0,377160,violent,no_multiplayer,more,positive,4
2,151603712,Fallout New Vegas,12.1,22380,violent,multiplayer,more,positive,3
3,151603712,Left 4 Dead 2,8.9,550,action,multiplayer,more,positive,2
4,151603712,HuniePop,8.5,339800,violent,no_multiplayer,more,positive,2


In [28]:
df_users.sort_values(by=['User_id'],inplace=True,ascending = True)

In [29]:
new_index = [ i for i in range(df_users.shape[0])]
df_users.index = new_index
df_users.head()

,User_id,Game,Hours-played,Game_id,Game_genre,Game_category,Game_playtime,Game_positive_votes,rating
0,5250,Dota 2,0.2,570,action,multiplayer,more,positive,1
1,5250,Team Fortress 2,0.8,440,action,multiplayer,more,positive,1
2,5250,Alien Swarm,4.9,630,action,multiplayer,more,positive,2
3,5250,Portal 2,13.6,620,action,multiplayer,more,positive,3
4,5250,Cities Skylines,144.0,255710,action,no_multiplayer,more,positive,5


In [30]:
reader = Reader()
data = Dataset.load_from_df(df_users[['User_id', 'Game_id', 'rating']], reader)

In [31]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0870  1.0974  1.0969  1.1007  1.1077  1.0979  0.0067  
MAE (testset)     0.8616  0.8724  0.8717  0.8722  0.8813  0.8718  0.0062  
Fit time          3.74    3.37    3.32    3.47    3.41    3.46    0.15    
Test time         0.21    0.09    0.13    0.10    0.09    0.12    0.05    


{'test_rmse': array([1.08698193, 1.09737916, 1.09693411, 1.10066944, 1.10767704]),
 'test_mae': array([0.86158114, 0.87243584, 0.87169811, 0.87224661, 0.88125453]),
 'fit_time': (3.741610527038574,
  3.3697242736816406,
  3.3225536346435547,
  3.4720897674560547,
  3.4078259468078613),
 'test_time': (0.21071171760559082,
  0.09057044982910156,
  0.12880873680114746,
  0.09670853614807129,
  0.08809041976928711)}

In [32]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [33]:
#take the user id and find his index
chosen_user = 151603712
user_to_idx = {user: i for i, user in enumerate(df_users.User_id.unique())}

In [34]:
df_users[df_users['User_id'] == 151603712]

,User_id,Game,Hours-played,Game_id,Game_genre,Game_category,Game_playtime,Game_positive_votes,rating
37008,151603712,The Elder Scrolls V Skyrim,273.0,72850,violent,no_multiplayer,more,positive,5
37009,151603712,Dragon Age Origins - Ultimate Edition,1.3,47810,violent,no_multiplayer,more,positive,1
37010,151603712,HuniePop,8.5,339800,violent,no_multiplayer,more,positive,2
37011,151603712,Dota 2,0.5,570,action,multiplayer,more,positive,1
37012,151603712,Left 4 Dead 2,8.9,550,action,multiplayer,more,positive,2
37013,151603712,Path of Exile,8.1,238960,violent,multiplayer,more,positive,2
37014,151603712,Fallout New Vegas,12.1,22380,violent,multiplayer,more,positive,3
37015,151603712,Robocraft,0.4,301520,violent,multiplayer,more,positive,1
37016,151603712,Garry's Mod,0.1,4000,action,multiplayer,more,positive,1
37017,151603712,Poly Bridge,7.5,367450,action,multiplayer,more,positive,2


In [35]:
# def svd_pred(user, trainSet, svd):
anti_testset_user = []
targetUser = user_to_idx[151603712] #inner_id of the target user
fillValue = trainset.global_mean
user_item_ratings = trainset.ur[targetUser]
user_items = [item for (item,_) in (user_item_ratings)]
ratings = trainset.all_ratings()
for iid in trainset.all_items():
    if(iid not in user_items):
        anti_testset_user.append((trainset.to_raw_uid(targetUser),trainset.to_raw_iid(iid),fillValue))

In [36]:
predictions = svd.test(anti_testset_user)
pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)
pred.head(10)

,uid,iid,r_ui,est,details
110,151603712,8930,2.254348,3.203586,{'was_impossible': False}
116,151603712,236430,2.254348,3.199529,{'was_impossible': False}
7,151603712,730,2.254348,3.190026,{'was_impossible': False}
436,151603712,250900,2.254348,3.171922,{'was_impossible': False}
111,151603712,292030,2.254348,3.116145,{'was_impossible': False}
3,151603712,325610,2.254348,3.053541,{'was_impossible': False}
112,151603712,105600,2.254348,3.002122,{'was_impossible': False}
115,151603712,287700,2.254348,2.984547,{'was_impossible': False}
227,151603712,48700,2.254348,2.961224,{'was_impossible': False}
493,151603712,313160,2.254348,2.893122,{'was_impossible': False}


In [37]:
rec_list = pred.head(10)['iid'].to_list()
rec_list

[8930, 236430, 730, 250900, 292030, 325610, 105600, 287700, 48700, 313160]

In [39]:
game_id = {}
games = df_users["Game"].value_counts()
for game in games.keys():
    game_id[game] = df_details.iloc[list(df_details["name"]).index(game)]["appid"]

In [40]:
idx_to_game = {idx: game for game, idx in game_id.items()}

In [41]:
for rec in rec_list:
    print(str(rec)+":",idx_to_game[rec])

8930: Sid Meier's Civilization V
236430: DARK SOULS II
730: Counter-Strike Global Offensive
250900: The Binding of Isaac Rebirth
292030: The Witcher 3 Wild Hunt
325610: Total War ATTILA
105600: Terraria
287700: METAL GEAR SOLID V THE PHANTOM PAIN
48700: Mount & Blade Warband
313160: Farming Simulator 15


## K-Nearest Neighbours item-oriented

In [42]:
from surprise import KNNBasic

In [43]:
sim_options = {'name': 'cosine', 'user_based': False} # compute  similarities between items
knn = KNNBasic(sim_options=sim_options)
# Run 5-fold cross-validation and print results
cross_validate(knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2871  1.3021  1.2940  1.3229  1.3038  1.3020  0.0121  
MAE (testset)     1.0180  1.0288  1.0311  1.0448  1.0348  1.0315  0.0087  
Fit time          0.66    0.70    0.66    0.67    0.65    0.67    0.02    
Test time         0.74    0.62    0.64    0.62    0.60    0.64    0.05    


{'test_rmse': array([1.28707959, 1.30210549, 1.2939843 , 1.32288933, 1.30380497]),
 'test_mae': array([1.01802549, 1.02876703, 1.03114244, 1.04477517, 1.03483325]),
 'fit_time': (0.6551821231842041,
  0.6953742504119873,
  0.6642239093780518,
  0.6740579605102539,
  0.6549746990203857),
 'test_time': (0.7381877899169922,
  0.6226339340209961,
  0.6442816257476807,
  0.6206028461456299,
  0.5964407920837402)}

In [44]:
predictions = knn.test(anti_testset_user)
pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)
pred.head(10)

,uid,iid,r_ui,est,details
2183,151603712,368650,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1103,151603712,335460,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1116,151603712,286930,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1923,151603712,276890,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1922,151603712,284020,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
2191,151603712,326380,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1114,151603712,274250,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1113,151603712,264120,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1921,151603712,284000,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
2190,151603712,320670,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"


In [45]:
rec_list = pred.head(10)['iid'].to_list()
rec_list

[368650,
 335460,
 286930,
 276890,
 284020,
 326380,
 274250,
 264120,
 284000,
 320670]

In [46]:
idx_to_game = {idx: game for game, idx in game_id.items()}

In [47]:
for rec in rec_list:
    print(str(rec)+":",idx_to_game[rec])

368650: Dark Echo
335460: An Assassin in Orlandes
286930: Crouching Pony Hidden Dragon
276890: FATE Undiscovered Realms
284020: Freddi Fish and Luther's Maze Madness
326380: Mystery of Neuschwanstein
274250: OlliOlli
264120: Victory The Age of Racing
284000: Putt-Putt and Pep's Balloon-o-Rama
320670: Season Match 3 - Curse of the Witch Crow


## K-Nearest Neighbours user-oriented

In [48]:
sim_options = {'name': 'cosine', 'user_based': True} # compute  similarities between users
knn = KNNBasic(sim_options=sim_options)
# Run 5-fold cross-validation and print results
cross_validate(knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1212  1.1099  1.1106  1.1182  1.1233  1.1166  0.0054  
MAE (testset)     0.8905  0.8787  0.8826  0.8880  0.8898  0.8859  0.0046  
Fit time          17.37   17.89   11.26   11.66   13.32   14.30   2.81    
Test time         2.87    2.72    1.78    2.02    1.94    2.27    0.44    


{'test_rmse': array([1.12115561, 1.10991633, 1.11057709, 1.1181509 , 1.12325875]),
 'test_mae': array([0.89051147, 0.87868409, 0.88259796, 0.88796965, 0.88980323]),
 'fit_time': (17.372502088546753,
  17.885334253311157,
  11.258788824081421,
  11.663550615310669,
  13.316982507705688),
 'test_time': (2.868731737136841,
  2.724318742752075,
  1.7829790115356445,
  2.016451835632324,
  1.944352149963379)}

In [49]:
predictions = knn.test(anti_testset_user)
pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)
pred.head(10)

,uid,iid,r_ui,est,details
1064,151603712,261470,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
2279,151603712,277450,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1158,151603712,289520,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1303,151603712,255260,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1327,151603712,15240,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1183,151603712,297060,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
875,151603712,3960,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1834,151603712,206040,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1489,151603712,890400,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"
1832,151603712,206020,2.254348,5.0,"{'actual_k': 1, 'was_impossible': False}"


In [50]:
rec_list = pred.head(10)['iid'].to_list()
rec_list

[261470, 277450, 289520, 255260, 15240, 297060, 3960, 206040, 890400, 206020]

In [51]:
idx_to_game = {idx: game for game, idx in game_id.items()}

In [52]:
for rec in rec_list:
    print(str(rec)+":",idx_to_game[rec])

261470: Distant Worlds Universe
277450: Imperial Glory
289520: King's Bounty Dark Side
255260: Pro Cycling Manager 2014
15240: Silent Hunter Wolves of the Pacific U-Boat Missions
297060: Freaking Meatbags
3960: Shattered Union
206040: Avernum 5
890400: Uncharted Waters Online
206020: Avernum 4
